In [2]:
# Read text file with a regex (1 or more spaces) as the separator.
import pandas as pd
text_attributes = pd.read_csv("../datasets/EFIGI_attributes.txt", sep = " +")

<ipython-input-2-8c2d9b619525>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  text_attributes = pd.read_csv("../datasets/EFIGI_attributes.txt", sep = " +")


In [3]:
text_attributes

,PGCname,T,T_inf,T_sup,Bulge_to_Total,Bulge_to_Total_inf,Bulge_to_Total_sup,Arm_Strength,Arm_Strength_inf,Arm_Strength_sup,...,Hot_Spots_sup,Inclination,Inclination_inf,Inclination_sup,Contamination,Contamination_inf,Contamination_sup,Multiplicity,Multiplicity_inf,Multiplicity_sup
0,PGC0000212,1,0,2,0.50,0.50,0.50,0.25,0.25,0.25,...,0.25,0.75,0.75,0.75,0.00,0.00,0.25,0.25,0.25,0.25
1,PGC0000218,0,-1,1,0.75,0.50,0.75,-1.00,-1.00,-1.00,...,0.00,1.00,1.00,1.00,0.25,0.25,0.25,0.00,0.00,0.00
2,PGC0000243,-2,-3,-2,1.00,0.75,1.00,0.00,0.00,0.25,...,0.25,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.00,0.00
3,PGC0000255,9,9,10,0.00,0.00,0.25,0.00,0.00,0.25,...,0.25,0.25,0.00,0.25,0.25,0.00,0.25,0.00,0.00,0.00
4,PGC0000281,7,6,9,0.25,0.00,0.50,0.00,0.00,0.25,...,0.75,0.25,0.00,0.25,0.50,0.50,0.50,0.25,0.00,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4453,PGC0072806,6,4,8,0.25,0.25,0.25,0.00,0.00,0.50,...,0.25,0.25,0.25,0.50,0.00,0.00,0.00,0.00,0.00,0.00
4454,PGC0072922,4,3,4,0.25,0.25,0.50,0.50,0.50,0.50,...,0.25,0.00,0.00,0.25,0.50,0.50,0.50,0.00,0.00,0.00
4455,PGC0072927,3,2,4,0.50,0.50,0.75,0.25,0.25,0.50,...,0.25,0.00,0.00,0.25,0.00,0.00,0.25,0.00,0.00,0.00
4456,PGC0072930,3,2,4,0.75,0.50,0.75,0.00,0.00,0.25,...,0.25,0.50,0.25,0.50,0.25,0.00,0.25,0.00,0.00,0.00


### We want to get more grannular with the target labels, so we will append a new columnn to the CSV to split up the types like such:

### 0: Elliptical
### 1: Lenticular
### 2: Spiral
### 3: Irregular
### 4: Dwarf Spheroid Elliptical

In [4]:
# Using a for loop on a Numpy array in this way is actually quite efficient.
# Source: https://stackoverflow.com/questions/35215161/most-efficient-way-to-map-function-over-numpy-array\
def type_splitter(n: int):
    if -6 <= n <= -4:
        n = 0
    elif -3 <= n <= -1:
        n = 1
    elif 0 <= n <= 9:
        n = 2
    elif n == 10:
        n = 3
    else:
        n = 4
    return n
    
def split_types_bycategory(type_list): 
    return [type_splitter(n) for n in type_list]

In [5]:
import numpy as np
# Append a new column to the CSV with the split data.
types_list_original = np.array(text_attributes["T"])
types_list_original

array([ 1,  0, -2, ...,  3,  3, 10], dtype=int64)

In [9]:
types_list_bycategory = split_types_bycategory(types_list_original)
text_attributes["category_label"] = types_list_bycategory
types_list_bycategory[0:10]

[2, 2, 1, 2, 2, 2, 2, 2, 2, 2]

In [10]:
efigi_labels = text_attributes[["PGCname", "T", "category_label"]]
efigi_labels

,PGCname,T,category_label
0,PGC0000212,1,2
1,PGC0000218,0,2
2,PGC0000243,-2,1
3,PGC0000255,9,2
4,PGC0000281,7,2
...,...,...,...
4453,PGC0072806,6,2
4454,PGC0072922,4,2
4455,PGC0072927,3,2
4456,PGC0072930,3,2


In [12]:
# Show value counts of processed labels
efigi_labels["category_label"].value_counts()

2    3315
1     537
0     289
3     248
4      69
Name: category_label, dtype: int64

In [13]:
# Save the parsed data to a CSV file.
efigi_labels.to_csv("EFIGI_labels.csv")

In [22]:
feature_data["label"] = efigi_labels.category_label

In [43]:
feature_data.to_csv("EFIGI_SIFT_feature_data.csv", index=False)